# Adaptive Partition-Based Level Decomposition Methods for Solving Two-Stage Stochastic Programs with Fixed Recourse
[W. van Ackooij](mailto:wim.van.ackooij@gmail.com), [W. de Oliveira](mailto:welington@ime.uerj.br) and [Yongjia Song](mailto:yongjis@clemson.edu)

In [1]:
# techincal lines
using LinearAlgebra
using JuMP
using Gurobi

#create gurobi environment
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only - expires 2021-05-08


In [2]:
#reading the data file
include("data.jl");

In [3]:
#define the model
MVP = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));

#define the variables
@variable(MVP,u[1:nbFirstVars]); #u is the 1st stage variable in the MVP
@variable(MVP,v[1:nbSecVars]);   #v is the 2nd stage variable in the MVP


#define the objective
@objective(MVP,Min,sum(objcoef[i]*u[i] for i=1:nbFirstVars)+sum(objcoef[nbFirstVars+j]*v[j] for j=1:nbSecVars));


#define the constraints

#the 1st satge variable is bigger/smaller than its lower/upper bound
for i=1:nbFirstVars
    @constraint(MVP, u[i] >=firstvarlb[i]);
    @constraint(MVP, u[i] <=firstvarub[i]);
end

#the 2nd satge variable is bigger/smaller than the expected value of its lower/upper bound
for i=1:nbSecVars
    @constraint(MVP, v[i] >= (sum(secondvarlb[k,i] for k=1:nbScens)/nbScens));
    @constraint(MVP, v[i] <= (sum(secondvarub[k,i] for k=1:nbScens)/nbScens));
end

#the lower/upper bound constraints related to the 1st satge variable 
for i=1:nbFirstRows
    @constraint(MVP, sum(u[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) >=firstconstrlb[i])    
    @constraint(MVP, sum(u[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) <=firstconstrub[i])    
end

#the bound constraints related to the 2nd satge variable 
for i=1:nbSecRows
    if secondconstrsense[i] == 0
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) == (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
    if secondconstrsense[i] == 1
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) >= (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
    if secondconstrsense[i] == -1
        @constraint(MVP, sum(u[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))+sum(v[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) <= (sum(secondconstrbd[j,i] for j=1:nbScens)/nbScens))
    end
end

In [4]:
optimize!(MVP)
status_mvp = termination_status(MVP);
if status_mvp != MOI.OPTIMAL
    println("Models status is: ", status_mvp, " === Oops! :/")
else
    obj_mvp = objective_value(MVP);
    xval = value.(u);
    yval = value.(v)
    println("Finish solving! Status = ", status_mvp);
    println("Optimal objective value = ", obj_mvp);
    println("x optimal value = ", xval);
    println("y optimal value = ", yval);
end

Finish solving! Status = OPTIMAL
Optimal objective value = 218.01398122003295
x optimal value = [0.0, 4.382227853768909, 0.0, 2.143650429190988, 0.15390412269472595, 0.0, 0.00907823792055301, 0.0, 0.4076869778625408, 0.0, 0.010530876431487727, 0.8259507917591504, 0.0, 0.0, 0.0, 0.07554298065344937, 0.1124766053641693, 0.29335021017563373, 0.1896740640303116, 0.13928779971105198]
y optimal value = [0.3701034309592, 0.4134238912422419, 2.245219202035589, 0.0, 0.0, 0.27798970024234154, 1.9257370492861778, 1.9669669988627463, 8.706661809364638, 2.872718146711897, 0.8105021826000701, 0.0, 2.8396558031285073, 0.0, 0.27609095167001063, 0.0, 2.2041596495119946, 0.17142382332988335, 0.0, 3.0010287475706714, 2.7729648641220646, 2.816580851704152, 2.11346308726969, 0.0, 0.0, 0.0, 7.054344486774584, 0.0, 0.0, 0.0]


In [5]:
#coarse cut
function SPC(xval,P,p)
    piC = zeros(p,nbSecRows+nbSecVars*2);
    QC = zeros(p);
    
    #For each cluster we will do the following 
    for k=1:p
        for i=1:nbSecVars
            #update the 2nd stage variable LB & UB
            TempRHSlb = sum(secondvarlb[j,i] for j in P[k])/length(P[k])
            set_normalized_rhs(constrlb[i], TempRHSlb);
            
            TempRHSub = sum(secondvarub[j,i] for j in P[k])/length(P[k])
            set_normalized_rhs(construb[i], TempRHSub);
        end
        
        #update the bound of each of the constraints 
        for i=1:nbSecRows
            if secondconstrsense[i] == 0
                TempRHS1 = sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]));
                set_normalized_rhs(constr1[i], TempRHS1);
            end
            
            if secondconstrsense[i] == 1
                TempRHS2 = sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]));
                set_normalized_rhs(constr2[i], TempRHS2);
            end
            if secondconstrsense[i] == -1
                TempRHS3 = sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]));
                set_normalized_rhs(constr3[i], TempRHS3);
            end
        end
        #We solve the subproblem & obtain its objective value
        optimize!(SubProblem)
        statusSP = termination_status(SubProblem);
        if statusSP != MOI.OPTIMAL
            println("Models status is: ", statusSP, " === Oops! :/")
            exit()
        else
            QC[k] = objective_value(SubProblem);
            
            #We get the dual multiplier of each constraint for this scenario 
            for i=1:nbSecVars
                if -1.0e10<secondvarlb[i]
                    piC[k,i] = dual(constrlb[i])
                end
                if 1.0e10>secondvarub[i]
                    piC[k,nbSecVars+i] = dual(construb[i])
                end
            end
            for i=1:nbSecRows
                if secondconstrsense[i] == 0
                    piC[k,nbSecVars+nbSecVars+i]=dual(constr1[i]);
                end
                if secondconstrsense[i] == 1
                    piC[k,nbSecVars+nbSecVars+i]=dual(constr2[i]);
                end
                if secondconstrsense[i] == -1
                    piC[k,nbSecVars+nbSecVars+i]=dual(constr3[i]);
                end  
            end
        end
    end
    return QC, piC
end

SPC (generic function with 1 method)

In [6]:
#coarse cut
function SPSC(xval,P,c)
    
    #For each scenario in the cluster we will do the following 
    piSC = zeros(nbScens,nbSecRows+nbSecVars*2);
    QSC = zeros(nbScens);
    #For each scenario we will do the following 
    for k in P[c]
        for i=1:nbSecVars
            #update the 2nd stage variable LB & UB
            TempRHSlb = secondvarlb[k,i]
            set_normalized_rhs(constrlb[i], TempRHSlb);
            TempRHSub = secondvarub[k,i]
            set_normalized_rhs(construb[i], TempRHSub);
        end
        
        #update the bound of each of the constraints 
        for i=1:nbSecRows
            if secondconstrsense[i] == 0
                TempRHS1 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr1[i], TempRHS1);
            end
            if secondconstrsense[i] == 1
                TempRHS2 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr2[i], TempRHS2);
            end
            if secondconstrsense[i] == -1
                TempRHS3 = secondconstrbd[k,i] - sum(xval[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))
                set_normalized_rhs(constr3[i], TempRHS3);
            end
        end
        #We solve the subproblen & obtain its objective value
        optimize!(SubProblem)
        statusSP = termination_status(SubProblem);
        if statusSP != MOI.OPTIMAL
            println("Models status is: ", statusSP, " === Oops! :/")
            exit()
        else
            QSC[k] = objective_value(SubProblem);
            #We get the dual multiplier of each constraint for this scenario 
            for i=1:nbSecVars
                piSC[k,i] = dual(constrlb[i])
                piSC[k,nbSecVars+i] = dual(construb[i])
            end
            for i=1:nbSecRows
                if secondconstrsense[i] == 0
                    piSC[k,nbSecVars+nbSecVars+i]=dual(constr1[i]);
                end
                if secondconstrsense[i] == 1
                    piSC[k,nbSecVars+nbSecVars+i]=dual(constr2[i]);
                end
                if secondconstrsense[i] == -1
                    piSC[k,nbSecVars+nbSecVars+i]=dual(constr3[i]);
                end  
            end
            
        end

    end
    return QSC, piSC
end

SPSC (generic function with 1 method)

In [7]:
function solve_master(LB,xval,thetaval)
    optimize!(Master)
    status_master = termination_status(Master);
    if status_master != MOI.OPTIMAL
        #println("Models status is: ", status_master, " === Oops! :/")
        #exit()
    else
        xval = value.(x);
        thetaval = value(theta);
        LB = sum(objcoef[i]*xval[i] for i=1:nbFirstVars)+thetaval
    end
    return status_master, LB, xval, thetaval
end

solve_master (generic function with 1 method)

In [8]:
 function Refine(P,PC,g,pi,ϵ,xxval,Q)
    Refine_Start=time();
    temp = [];
    reps = [];
    reps_indc = [];
    push!(temp,[PC[g][1]])
    push!(reps,pi[PC[g][1],:]);
    push!(reps_indc,PC[g][1]);
    
    for k=2:length(PC[g])
        count = 0;
        for s=1:length(temp)
            diff = pi[PC[g][k],:]-reps[s];
            if norm(diff) < ϵ 
                push!(temp[s],PC[g][k])
                break
            else
                #check if the solution is degenerate solutions
                obj_diff = Q[PC[g][k]] - Q[reps_indc[s]]
                if abs(obj_diff)/abs(Q[reps_indc[s]]) < ϵ
                    push!(temp[s],PC[g][k])
                    break
                else
                    count +=1
                end
            end
        end
        if count == length(temp)
            push!(temp,[PC[g][k]])
            push!(reps, pi[PC[g][k],:]);
            push!(reps_indc, PC[g][k]);
        end
    end 
    
    filter!(e->e!=PC[g],P)
    for i=1:length(temp)
        push!(P,temp[i])
    end
    
    elaps = time()-Refine_Start;
    return P, elaps
end

Refine (generic function with 1 method)

In [9]:
# A function which defines the level parameter
f_level(f_low,f_up,γ) = f_low+γ*(f_up-f_low);

In [10]:
#define the master problem
Master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
@variable(Master,x[1:nbFirstVars]);
@variable(Master,theta);
@variable(Master,z[1:nbFirstVars]);
@objective(Master,Min,sum(objcoef[i]*x[i] for i=1:nbFirstVars)+theta);

#To prevent our Master problem solution in the first iteration from being unbounded since we don't have a bound on theta, we add a constraint were our objective is bounded by the MVP objective 
@constraint(Master, sum(objcoef[i]*x[i] for i=1:nbFirstVars)+theta>=getobjectivevalue(MVP));

#lower/upper bound on the 1st stage variable 
for i=1:nbFirstVars
    if -1.0e10<firstvarlb[i]
        @constraint(Master, x[i] >=firstvarlb[i]);
    end
    if 1.0e10>firstvarub[i]
        @constraint(Master, x[i] <=firstvarub[i]);
    end
end

#the lower/upper bound constraints related to the 1st satge variable 
for i=1:nbFirstRows
    @constraint(Master, sum(x[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) >=firstconstrlb[i]);    
    @constraint(Master, sum(x[firstconstrind[i][j]]*firstconstrcoef[i][j] for j=1:length(firstconstrcoef[i])) <=firstconstrub[i]);    
end

level_cons = @constraint(Master, sum(objcoef[i]*x[i] for i=1:nbFirstVars)+theta <= 0);

In [11]:
# define the second stage subproblem
SubProblem = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
@variable(SubProblem,y[1:nbSecVars]);
@objective(SubProblem,Min,sum(objcoef[nbFirstVars+j]*y[j] for j=1:nbSecVars));

#We will initilaize the RHS of all the 2nd stage constraints to be zero and we will change it later
@constraint(SubProblem,constrlb[i=1:nbSecVars], y[i] >= 0);
@constraint(SubProblem,construb[i=1:nbSecVars], y[i] <= 0);
@constraint(SubProblem,constr1[i=1:nbSecRows; secondconstrsense[i] == 0], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) == 0);
@constraint(SubProblem,constr2[i=1:nbSecRows; secondconstrsense[i] == 1], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) >= 0);
@constraint(SubProblem,constr3[i=1:nbSecRows; secondconstrsense[i] == -1], sum(y[second2ndconstrind[i][j]]*second2ndconstrcoef[i][j] for j=1:length(second2ndconstrcoef[i])) <= 0);

In [12]:
#initialize stuff
P = [[k for k=1:nbScens]]; #The initial partition
PC = deepcopy(P); #a copy of the partition
p = length(P); 

LB = obj_mvp;
UB = 1e10;

ϵ = 1e-5;
iter = 0;

thetaval = -1e10

x_incumbent = deepcopy(xval); #initialize the incumbent

γ = 0.2929;

f_low = LB;
f_up = UB;


#the normalizing constraints
for i=1:nbFirstVars
    @constraint(Master, x[i]-x_incumbent[i] <=z[i]);
    @constraint(Master, x_incumbent[i]-x[i] <=z[i]);
end


start=time();
while (UB-LB)*1.0/max(1e-10,abs(UB)) > 1e-4
    
    
    #First set the objective using the incumbent
    @objective(Master,Min,sum(objcoef[i]*x[i] for i=1:nbFirstVars)+theta+1/2*sum(z[i] for i=1:nbFirstVars));
    #Define the level set paramter and update the constraint
    f_val = f_level(f_low,f_up,γ);
    set_normalized_rhs(level_cons, f_val);
    
    status_master, LB, xval, thetaval = solve_master(LB,xval,thetaval);
    
    if status_master == MOI.INFEASIBLE_OR_UNBOUNDED
        println("Master problem status is: ", status_master, " === Oops! :/")
        set_optimizer_attribute(farmer_master, "DualReductions", 0)
    elseif status_master == MOI.INFEASIBLE 
        #update the level set and break
        println("The level set is empty ==> update f_low")
        println("f_val = ", f_val)
        f_low = f_val;
    elseif status_master == MOI.OPTIMAL
        iter = iter + 1;
        #attempt to generate a coarse cut
        QC, piC = SPC(xval,P,p);
        θtemp = sum(QC[s]*(length(P[s])/nbScens) for s=1:p);    
        if length(PC) == nbScens
            UB = min(sum(objcoef[i]*xval[i] for i=1:nbFirstVars)+θtemp,UB);
        end

        if (θtemp-thetaval)/max(1e-10,abs(thetaval)) > ϵ
            @constraint(Master,
                        theta>= 
                        sum((sum(piC[k,i]*sum(secondvarlb[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                       +sum(piC[k,nbSecVars+i]*sum(secondvarub[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                       +sum(piC[k,nbSecVars+nbSecVars+i]*(sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) 
                       -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows))*(length(P[k])/nbScens) for k=1:p));
        else
            #generate a semi_coarse cut

            tempUB = 0;
            for c=1:length(PC)
                QSC, piSC = SPSC(xval,P,c);
                θtemp = sum(QSC[k] for k in PC[c])*(1/nbScens)
                tempUB += θtemp; 
                if length(PC) > 1
                    θtemp += sum(QC[k]*(length(PC[k])/nbScens) for k=1:p if k != c);
                end
                if c == length(PC)
                    UB = min(sum(objcoef[i]*xval[i] for i=1:nbFirstVars)+tempUB,UB);
                end


                if (θtemp-thetaval)/max(1e-10,abs(thetaval)) > ϵ
                    if p == length(PC)
                        @constraint(Master,
                            theta>= 
                            sum(sum(piSC[k,i]*secondvarlb[k,i] for i=1:nbSecVars)
                           +sum(piSC[k,nbSecVars+i]*secondvarub[k,i] for i=1:nbSecVars)
                           +sum(piSC[k,nbSecVars+nbSecVars+i]*(secondconstrbd[k,i] 
                           -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows) for k in P[c])*(1/nbScens)
                                    );
                    else
                        @constraint(Master,
                            theta>= 
                            sum(sum(piSC[k,i]*secondvarlb[k,i] for i=1:nbSecVars)
                           +sum(piSC[k,nbSecVars+i]*secondvarub[k,i] for i=1:nbSecVars)
                           +sum(piSC[k,nbSecVars+nbSecVars+i]*(secondconstrbd[k,i] 
                           -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows) for k in P[c])*(1/nbScens)
                           +sum((sum(piC[k,i]*sum(secondvarlb[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                           +sum(piC[k,nbSecVars+i]*sum(secondvarub[s,i] for s in P[k])/length(P[k]) for i=1:nbSecVars)
                           +sum(piC[k,nbSecVars+nbSecVars+i]*(sum(secondconstrbd[s,i] for s in P[k])/length(P[k]) 
                           -sum(x[second1stconstrind[i][j]]*second1stconstrcoef[i][j] for j=1:length(second1stconstrcoef[i]))) for i=1:nbSecRows))*(length(P[k])/nbScens) for k=1:p if k != c)         
                                    );
                    end

                    if length(PC[c]) > 1
                        #P, elaps = Refine(P,PC,c,piSC,ϵ,xval,QSC);
                        P, elaps = Refine(P,PC,c,piSC,1e-15,xval,QSC);
                    end
                    P = sort!(P, by=length, rev=true);
                    PC = deepcopy(P);
                    p = length(P);
                    break
                end
            end

        end
        x_incumbent = deepcopy(xval);
        f_low = LB;
        f_up = UB;
        
        #given the new dual multiplier, refine the partition
        #check to make sure # of clusters < the total # of scenarios
        println("iter = ", iter);
        println("LB = ", LB);
        println("UB = ", UB);
        println("===============================================================");        
    else
        println("Master problem status is: ", status_master, " === Oops! :/")
        break
    end

end
Elapsed = time()-start;
println("******************************************")
println("******************************************")
println("Problem is solved! Yaay")
println("******************************************")
println("Elapsed time = ", Elapsed, " seconds")
println("iter = ", iter);
println("# of clusters used  = ", p);
println("LB = ", LB);
println("UB = ", UB);
println("xvalue = ", xval);

iter = 1
LB = 218.01398122003295
UB = 272.48302690625263
iter = 2
LB = 218.0139812200329
UB = 272.48302690625263
iter = 3
LB = 218.01398122003295
UB = 272.48302690625263
iter = 4
LB = 218.01398122003295
UB = 272.48302690625263
The level set is empty ==> update f_low
f_val = 233.96796470152668
iter = 5
LB = 234.7635694635517
UB = 272.48302690625263
iter = 6
LB = 237.45342141120366
UB = 272.48302690625263
iter = 7
LB = 237.2801882308054
UB = 272.48302690625263
iter = 8
LB = 237.34633238443843
UB = 272.48302690625263
iter = 9
LB = 237.36052416172112
UB = 272.48302690625263
iter = 10
LB = 237.36628913483844
UB = 272.48302690625263
iter = 11
LB = 237.36693650894793
UB = 272.48302690625263
iter = 12
LB = 237.36693650894793
UB = 272.48302690625263
iter = 13
LB = 241.69170693655428
UB = 272.48302690625263
iter = 14
LB = 242.7161583231281
UB = 272.48302690625263
iter = 15
LB = 243.25086792098764
UB = 272.48302690625263
iter = 16
LB = 243.77517222935765
UB = 272.48302690625263
iter = 17
LB = 243

iter = 68
LB = 247.85882409963395
UB = 272.48302690625263
iter = 69
LB = 247.85882409963395
UB = 272.48302690625263
iter = 70
LB = 247.87269799214766
UB = 272.48302690625263
iter = 71
LB = 247.87806432523894
UB = 272.48302690625263
iter = 72
LB = 247.89077926683186
UB = 272.48302690625263
iter = 73
LB = 247.90704470402468
UB = 272.48302690625263
iter = 74
LB = 247.9078862814338
UB = 272.48302690625263
iter = 75
LB = 247.90913308560965
UB = 272.48302690625263
iter = 76
LB = 247.90913308560965
UB = 272.48302690625263
iter = 77
LB = 247.9214096103434
UB = 272.48302690625263
iter = 78
LB = 247.9496849963817
UB = 272.48302690625263
iter = 79
LB = 247.9509889579877
UB = 272.48302690625263
iter = 80
LB = 247.95110007537036
UB = 272.48302690625263
iter = 81
LB = 247.95110007537036
UB = 272.48302690625263
iter = 82
LB = 248.07433849794464
UB = 272.48302690625263
iter = 83
LB = 248.23342616486607
UB = 272.48302690625263
iter = 84
LB = 248.289318351743
UB = 272.48302690625263
iter = 85
LB = 248.3

iter = 136
LB = 249.86974741624348
UB = 272.48302690625263
iter = 137
LB = 249.7393598365971
UB = 272.48302690625263
iter = 138
LB = 249.83257958993474
UB = 272.48302690625263
iter = 139
LB = 249.87893063871056
UB = 272.48302690625263
iter = 140
LB = 249.88538231765347
UB = 272.48302690625263
iter = 141
LB = 249.87471991676142
UB = 272.48302690625263
iter = 142
LB = 249.87793138297533
UB = 272.48302690625263
iter = 143
LB = 249.8974338195409
UB = 272.48302690625263
iter = 144
LB = 249.8898760813959
UB = 272.48302690625263
iter = 145
LB = 249.8898760813959
UB = 272.48302690625263
iter = 146
LB = 249.99205872403886
UB = 272.48302690625263
iter = 147
LB = 249.95417269421586
UB = 272.48302690625263
iter = 148
LB = 250.0325293068002
UB = 272.48302690625263
iter = 149
LB = 249.99590142136068
UB = 272.48302690625263
iter = 150
LB = 250.02517427962127
UB = 272.48302690625263
iter = 151
LB = 250.03294153726623
UB = 272.48302690625263
iter = 152
LB = 250.03294153726623
UB = 272.48302690625263
it

iter = 203
LB = 250.11008973930885
UB = 272.48302690625263
iter = 204
LB = 250.16740568917928
UB = 272.48302690625263
iter = 205
LB = 250.1207551908358
UB = 272.48302690625263
iter = 206
LB = 250.13292737731715
UB = 272.48302690625263
iter = 207
LB = 250.13958027571164
UB = 272.48302690625263
iter = 208
LB = 250.1258889866838
UB = 272.48302690625263
iter = 209
LB = 250.13037691322785
UB = 272.48302690625263
iter = 210
LB = 250.13037691322785
UB = 272.48302690625263
iter = 211
LB = 250.1384923321995
UB = 272.48302690625263
iter = 212
LB = 250.125513386953
UB = 272.48302690625263
iter = 213
LB = 250.12551338695295
UB = 272.48302690625263
iter = 214
LB = 250.12164538653295
UB = 272.48302690625263
iter = 215
LB = 250.12164538653295
UB = 272.48302690625263
iter = 216
LB = 250.13695733000895
UB = 272.48302690625263
iter = 217
LB = 250.13358366136265
UB = 272.48302690625263
iter = 218
LB = 250.13358366136265
UB = 272.48302690625263
iter = 219
LB = 250.12167938107885
UB = 272.48302690625263
it

iter = 270
LB = 250.34858366496258
UB = 272.48302690625263
iter = 271
LB = 250.3467346612285
UB = 272.48302690625263
iter = 272
LB = 250.3467346612285
UB = 272.48302690625263
iter = 273
LB = 250.34195037726326
UB = 272.48302690625263
iter = 274
LB = 250.3388937594121
UB = 272.48302690625263
iter = 275
LB = 250.34850189535157
UB = 272.48302690625263
iter = 276
LB = 250.34850189535157
UB = 272.48302690625263
iter = 277
LB = 250.32784079333942
UB = 272.48302690625263
iter = 278
LB = 250.32660350038375
UB = 272.48302690625263
iter = 279
LB = 250.34769257606504
UB = 272.48302690625263
iter = 280
LB = 250.3684994296962
UB = 272.48302690625263
iter = 281
LB = 250.36313958328728
UB = 272.48302690625263
iter = 282
LB = 250.36313958328728
UB = 272.48302690625263
iter = 283
LB = 250.35883635605745
UB = 272.48302690625263
iter = 284
LB = 250.35883635605745
UB = 272.48302690625263
iter = 285
LB = 250.35702145781
UB = 272.48302690625263
iter = 286
LB = 250.35702145781
UB = 272.48302690625263
iter = 

iter = 337
LB = 250.42400313131972
UB = 272.48302690625263
iter = 338
LB = 250.41723380192408
UB = 272.48302690625263
iter = 339
LB = 250.4156994249368
UB = 272.48302690625263
iter = 340
LB = 250.4156994249368
UB = 272.48302690625263
iter = 341
LB = 250.41045088331137
UB = 272.48302690625263
iter = 342
LB = 250.40669907648567
UB = 272.48302690625263
iter = 343
LB = 250.40669907648567
UB = 272.48302690625263
iter = 344
LB = 250.3997958525685
UB = 272.48302690625263
iter = 345
LB = 250.40875353062
UB = 272.48302690625263
iter = 346
LB = 250.40875353062
UB = 272.48302690625263
iter = 347
LB = 250.49164193440387
UB = 272.48302690625263
iter = 348
LB = 250.5619955364995
UB = 272.48302690625263
iter = 349
LB = 250.51014279687377
UB = 272.48302690625263
iter = 350
LB = 250.43225538005572
UB = 272.48302690625263
iter = 351
LB = 250.46173620049908
UB = 272.48302690625263
iter = 352
LB = 250.48330609785768
UB = 272.48302690625263
iter = 353
LB = 250.46550640197253
UB = 272.48302690625263
iter = 

iter = 404
LB = 250.5070211592339
UB = 272.48302690625263
iter = 405
LB = 250.5007111961033
UB = 272.48302690625263
iter = 406
LB = 250.5007111961033
UB = 272.48302690625263
iter = 407
LB = 250.51856304749856
UB = 272.48302690625263
iter = 408
LB = 250.5212831752955
UB = 272.48302690625263
iter = 409
LB = 250.51625074970443
UB = 272.48302690625263
iter = 410
LB = 250.51625074970443
UB = 272.48302690625263
iter = 411
LB = 250.56388613020394
UB = 272.48302690625263
iter = 412
LB = 250.5352467599155
UB = 272.48302690625263
iter = 413
LB = 250.5088331576367
UB = 272.48302690625263
iter = 414
LB = 250.5088331576367
UB = 272.48302690625263
iter = 415
LB = 250.52911413438858
UB = 272.48302690625263
iter = 416
LB = 250.52911413438858
UB = 272.48302690625263
iter = 417
LB = 250.51368605855296
UB = 272.48302690625263
iter = 418
LB = 250.56453126504783
UB = 272.48302690625263
iter = 419
LB = 250.54237211794796
UB = 272.48302690625263
iter = 420
LB = 250.54237211794796
UB = 272.48302690625263
iter

iter = 471
LB = 250.55610712371336
UB = 272.48302690625263
iter = 472
LB = 250.55790140684329
UB = 272.48302690625263
iter = 473
LB = 250.55790140684329
UB = 272.48302690625263
iter = 474
LB = 250.5606728066615
UB = 272.48302690625263
iter = 475
LB = 250.5606728066615
UB = 272.48302690625263
iter = 476
LB = 250.5690202363447
UB = 272.48302690625263
iter = 477
LB = 250.5690202363447
UB = 272.48302690625263
iter = 478
LB = 250.54717997326009
UB = 272.48302690625263
iter = 479
LB = 250.57080895188594
UB = 272.48302690625263
iter = 480
LB = 250.57080895188594
UB = 272.48302690625263
iter = 481
LB = 250.56133787038925
UB = 272.48302690625263
iter = 482
LB = 250.56133787038925
UB = 272.48302690625263
iter = 483
LB = 250.55141795941154
UB = 272.48302690625263
iter = 484
LB = 250.55141795941154
UB = 272.48302690625263
iter = 485
LB = 250.583657416965
UB = 272.48302690625263
iter = 486
LB = 250.57771872755052
UB = 272.48302690625263
iter = 487
LB = 250.5719076548664
UB = 272.48302690625263
iter